# Week 5 Homework: Refining our COMPAS Model

This week's homework is focused on making further improvements to our COMPAS-like deep neural network. We will be performing hyperparameter tuning. As discussed during lecture, you might not always see huge improvements from regularization. Oftentimes, this is because **hyperparameters** including regularization strengths are not optimal. Hyperparameter tuning trains several different models, each with a different combinations of hyperparameter values.

## Setting up the Environment

Run any code below by highlighting it and hitting `Shift + Enter`. Import the libraries below.

In [0]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers
from keras import backend as K
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import random
import requests
import zipfile
import io

# Fix random seed for reproducibility.
np.random.seed(1337)

## Read and Clean in Data

In [0]:
# Download and extract data.
r = requests.get("http://web.stanford.edu/class/cs21si/resources/unit3_resources.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

data = pd.read_csv("unit3_resources/compas-scores.csv", header = 0)

# Select fields we want.
fields_of_interest = ['name', 'sex', 'age', 'race', 'priors_count', 'c_charge_desc', 
                      'v_decile_score', 'decile_score', 'is_violent_recid', 'is_recid']
data = data[fields_of_interest]
# More interpretable column names.
data.columns = ['name', 'sex', 'age', 'race', 'num_priors', 'charge', 
                'violence_score', 'recidivism_score', 'violence_true', 'recidivism_true']

# Remove records with missing scores.
data = data.loc[(data.violence_score != -1) & (data.recidivism_score != -1)]
data = data.loc[(data.violence_true != -1) & (data.recidivism_true != -1)]

# Convert strings to numerical values.
sex_classes = {'Male': 0, 'Female' : 1}

processed_data = data.copy()
processed_data['sex'] = data['sex'].apply(lambda x: sex_classes[x])

# One-hot encode race.
processed_data = pd.get_dummies(processed_data, columns = ['race'])
columns = processed_data.columns.tolist()
columns = columns[0:3] + columns[9:] + columns[3:9]
processed_data = processed_data.reindex(columns = columns)

processed_data.head()

In [0]:
# Convert pandas dataframe to numpy array for easier processing.
processed_data = processed_data.values

## Partition into Train and Test Sets

This was all code you wrote during lecture, so we've given it to you as a freebie here!

In [0]:
# split into input (X) and output (Y) variables
X = processed_data[:,1:10] # sex, age, race, num_priors
y = processed_data[:,14] # recidivism_true

num_train = int(math.ceil(X.shape[0]*0.8))
num_test = int(math.floor(X.shape[0]*0.2))

X_train = X[:num_train]
y_train = y[:num_train]

X_test = X[num_train:]
y_test = y[num_train:]

num_classes = 2
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(X_train.shape[0], 'records in train set')
print(X_test.shape[0], 'records in test set')
print(X.shape[0], 'records in total')

## Set up our Evaluation Pipeline

Again, we wrote this together in class, so it's given to you here.

In [0]:
#########################################################
# Trains and evaluates given model. Returns loss and 
# accuracy.
#########################################################
def eval(model, verb = 2):
    # fit the model
    model.fit(X_train, y_train, 
              epochs = 30, 
              batch_size = batch_size,          
              validation_split = 0.1,
              verbose = verb,
              shuffle = False)
    
    # Evaluate the model.
    scores = model.evaluate(X_test, y_test)
    
    return scores

## Part 1: Create and Evaluate your Own Classifier

During lecture, you learned about adding `Dense` layers (which are just fully-connected layers that can have the parameter `kernel_regularizer`) to a Keras model. Check out this [guide](https://keras.io/getting-started/sequential-model-guide/) for some good examples in case you want a refresher.

**Your task:** Now that you've gotten your feet wet with Keras, in this exercise, you get to decide how you want to set up your model! Add layers of any size (paired with any activation functions) as you see fit, and if you want to experiment with other Keras add-ons that we haven't discussed before like `Dropout` or `BatchNormalization`, then go for it. Similarly, if you elect not to use L2-regularization, then that's up to you too. Good luck!

In [0]:
batch_size = 64
num_classes = 2

learning_rate = 2e-3
reg_strength = 1e-4

#########################################################
# Initializes neural network with dropout.
#########################################################
def nn_classifier(learning_rate, reg_strength, dropout_strength=0.5):
    # create model
    model = Sequential()

    # YOUR CODE HERE:

    
    
    
    # END CODE
    
    # compile model
    sgd = keras.optimizers.SGD(lr = learning_rate)
    model.compile(loss = keras.losses.categorical_crossentropy, 
                  optimizer = sgd, metrics=['accuracy'])
    
    return model

# Evaluate your model
for learning_rate in [1e-2]:
  for reg_strength in [1e-4]:
    print("Using learning rate %f and regularization strength %f..." % (learning_rate, reg_strength))
    model = nn_classifier(learning_rate, reg_strength)
    loss, acc = eval(model, verb = 2)
    print('\n\nTest loss:', loss)
    print('Test accuracy:', acc)

## Part 2: Hyperparameter Tuning

For the purposes of playing around with hyperparameter tuning and understanding the motivations behind it, we're going to be working with a naive implementation below.  

**Your task**: Tuning the hyperparameters and developing intuition for how they affect the final performance is a large part of using neural networks, so we want you to get a lot of practice. Below, you should experiment with different values of the various hyperparameters, including learning rate and regularization strength. Your goal in this exercise is to get as good of a result on the COMPAS dataset as you can, with a fully-connected deep neural network. Feel free to change the model you initialized above in Exercise 1 as well. The starter code below is there to give you a naive implementation of hyperparameter tuning. You can change it as you wish. **Challenge yourself to hit at least 69% validation accuracy with your best model.** Good luck!

In [0]:
def tune_hyperparams():
    best_model = (None, None, None)
    running_best_accuracy = 0

    # Play with these!
    learning_rate = [1e-2, 1e-3]
    reg_strength = [1e-3, 1e-4] 
    
    for lr in learning_rate:
        for reg in reg_strength:
            model = nn_classifier(lr, reg)
            model_loss, model_acc = eval(model, verb = 0)

            print('\n val_acc: {:f}, lr: {:f}, reg: {:f}\n'.format(
                    model_acc, lr, reg))

            if model_acc > running_best_accuracy:
                model_params = {"lr": lr, "reg": reg}
                best_model = (model, model_acc, model_params)
                running_best_accuracy = model_acc
            
    return best_model
        
best_model = tune_hyperparams()
print("\n\nBest Model Performance: ", best_model[1])
print("Hyperparameters of Best Model: ", best_model[2])